In [1]:
#XG BOOSTING>>>>>>>>>>>>>>>>>>>>>>>>>
import xgboost as xgb
import sklearn
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from imblearn.combine import SMOTEENN
import pickle

print(sklearn.__version__)

1.6.0


In [2]:
df=pd.read_csv("tele_churn_final.csv")
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1-12,tenure_group_13-24,tenure_group_25-36,tenure_group_37-48,tenure_group_49-60,tenure_group_61-72
0,0,0,29.85,29.85,0,True,False,False,True,True,...,False,False,True,False,True,False,False,False,False,False
1,1,0,56.95,1889.50,0,False,True,True,False,True,...,False,False,False,True,False,False,True,False,False,False
2,2,0,53.85,108.15,1,False,True,True,False,True,...,False,False,False,True,True,False,False,False,False,False
3,3,0,42.30,1840.75,0,False,True,True,False,True,...,True,False,False,False,False,False,False,True,False,False
4,4,0,70.70,151.65,1,True,False,True,False,True,...,False,False,True,False,True,False,False,False,False,False


In [3]:
df=df.drop('Unnamed: 0',axis=1)
df

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1-12,tenure_group_13-24,tenure_group_25-36,tenure_group_37-48,tenure_group_49-60,tenure_group_61-72
0,0,29.85,29.85,0,True,False,False,True,True,False,...,False,False,True,False,True,False,False,False,False,False
1,0,56.95,1889.50,0,False,True,True,False,True,False,...,False,False,False,True,False,False,True,False,False,False
2,0,53.85,108.15,1,False,True,True,False,True,False,...,False,False,False,True,True,False,False,False,False,False
3,0,42.30,1840.75,0,False,True,True,False,True,False,...,True,False,False,False,False,False,False,True,False,False
4,0,70.70,151.65,1,True,False,True,False,True,False,...,False,False,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,84.80,1990.50,0,False,True,False,True,False,True,...,False,False,False,True,False,True,False,False,False,False
7028,0,103.20,7362.90,0,True,False,False,True,False,True,...,False,True,False,False,False,False,False,False,False,True
7029,0,29.60,346.45,0,True,False,False,True,False,True,...,False,False,True,False,True,False,False,False,False,False
7030,1,74.40,306.60,1,False,True,False,True,True,False,...,False,False,False,True,True,False,False,False,False,False


In [4]:
x=df.drop('Churn',axis=1)
x
train_columns = x.columns
#pickle.dump(train_columns, open("model_columns.pkl", "wb"))
import joblib

joblib.dump(train_columns, "model_columns.pkl")

['model_columns.pkl']

In [5]:
y=df['Churn']
y

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64

In [6]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [7]:
model = xgb.XGBClassifier(n_estimators=20, max_depth=5, learning_rate=0.1, random_state=42)

In [8]:

model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 80.03%
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.91      0.87      1033
           1       0.67      0.50      0.57       374

    accuracy                           0.80      1407
   macro avg       0.75      0.70      0.72      1407
weighted avg       0.79      0.80      0.79      1407



In [9]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [10]:
xgb_model = xgb.XGBClassifier(n_estimators=200, max_depth=7, learning_rate=0.1, reg_alpha=0.1, reg_lambda=1.0, random_state=42)


In [11]:
xgb_model.fit(xr_train,yr_train)
yr_predict = xgb_model.predict(xr_test)
model_score_r = xgb_model.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9572649572649573
              precision    recall  f1-score   support

           0       0.94      0.97      0.95       508
           1       0.98      0.95      0.96       662

    accuracy                           0.96      1170
   macro avg       0.96      0.96      0.96      1170
weighted avg       0.96      0.96      0.96      1170



saving the model

In [12]:

import pickle
import joblib

In [13]:
 #with open('model.pkl','wb')as file:
 #   pickle.dump(xgb_model, file)

In [14]:
#load_model = pickle.load(open('model.pkl', 'rb'))

In [15]:
#model_score_r1 = load_model.score(xr_test, yr_test)

In [16]:
#print(model_score_r1)

In [17]:
joblib.dump(model, 'model.pkl')

['model.pkl']

In [18]:
joblib.dump(train_columns, 'model_columns.pkl')

['model_columns.pkl']